<a href="https://colab.research.google.com/github/IshaanKetchup/ASLtoEnglish/blob/main/ResNet18_on_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        #self.rl1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        if stride!=1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = torch.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = torch.relu(out)
        return out


class ResNet18(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()

        self.layer1 = nn.Sequential(
            ResidualBlock(64, 64, stride=2)
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128, stride=2)
        )
        self.layer3 = nn.Sequential(
            ResidualBlock(128, 256, stride=2)
        )
        self.layer4 = nn.Sequential(
            ResidualBlock(256, 512, stride=2)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out
model = ResNet18(num_classes = 10)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Define the data transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(32),  # CIFAR-10 images are 32x32
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:03<00:00, 45147657.23it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the training function
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 100 == 99:    # Print every 100 batches
            print(f'Epoch: {epoch}, Batch: {batch_idx+1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

# Define the test function
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
    print(f'Test Loss: {test_loss / len(test_loader.dataset):.3f}, Accuracy: {100. * correct / len(test_loader.dataset):.3f}%')


In [ ]:
# Initialize the model, criterion, and optimizer
model = ResNet18(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    train(model, train_loader, criterion, optimizer, epoch)
    test(model, test_loader, criterion)

# Save the model
torch.save(model.state_dict(), 'resnet18_cifar10.pth')


Epoch: 0, Batch: 100, Loss: 2.224
Epoch: 0, Batch: 200, Loss: 1.872
Epoch: 0, Batch: 300, Loss: 1.808
Test Loss: 0.014, Accuracy: 36.580%
Epoch: 1, Batch: 100, Loss: 1.713
Epoch: 1, Batch: 200, Loss: 1.675
Epoch: 1, Batch: 300, Loss: 1.654
Test Loss: 0.013, Accuracy: 41.070%
Epoch: 2, Batch: 100, Loss: 1.587
Epoch: 2, Batch: 200, Loss: 1.581
Epoch: 2, Batch: 300, Loss: 1.547
Test Loss: 0.012, Accuracy: 43.300%
Epoch: 3, Batch: 100, Loss: 1.492
Epoch: 3, Batch: 200, Loss: 1.475
Epoch: 3, Batch: 300, Loss: 1.465
Test Loss: 0.012, Accuracy: 47.130%
Epoch: 4, Batch: 100, Loss: 1.429
Epoch: 4, Batch: 200, Loss: 1.422
Epoch: 4, Batch: 300, Loss: 1.388
Test Loss: 0.012, Accuracy: 46.950%
Epoch: 5, Batch: 100, Loss: 1.380
Epoch: 5, Batch: 200, Loss: 1.368
Epoch: 5, Batch: 300, Loss: 1.365
Test Loss: 0.011, Accuracy: 51.220%
Epoch: 6, Batch: 100, Loss: 1.329
Epoch: 6, Batch: 200, Loss: 1.330
Epoch: 6, Batch: 300, Loss: 1.299
Test Loss: 0.011, Accuracy: 52.610%
Epoch: 7, Batch: 100, Loss: 1.311
